In [1]:
import psycopg2
from psycopg2 import OperationalError

import pandas as pd
import numpy as np
import matplotlib.pylab as plt

from IPython.display import HTML

***
    
    Ligação à base de dados

***

In [2]:
conn = psycopg2.connect(
    host = "contratos-base-gov1.cf87yxnqgph8.eu-central-1.rds.amazonaws.com",
    port = 5432,
    #database = "contratosbasegov",
    user = "contratosbasegov",
    password = "8n9nyeTBFUyCcLJShNrZdPUai2KQkue4")

cur = conn.cursor()

***

    Executar este comando quando ocorrer um erro ao tentar realizar uma query

***

In [3]:
cur.execute('''ROLLBACK;''')

***

# Funções


***

    ID's dos contratos

In [4]:
def all_ids(table):
    '''
    Função que retorna todos os ids dos contratos de uma tabela

    Parâmetros :    
        - table : tabela de interesse

    Return : 
        - list : ids de todos os contratos de uma tabela 
    '''

    cur = conn.cursor()
    cur.execute(''' 
                SELECT id
                FROM "{}"; '''.format(table)) 
    return list(cur.fetchall())


ids = all_ids("contratos")

    Nome das colunas

In [5]:
def col_names(table):
    '''
    Função que retorna os nomes das colunas de uma tabela
    
    Parâmetros : 
        - table : tabela de interesse

    return : 
        - pandas DataFrame : nomes das colunas
    '''

    cur = conn.cursor()
    cur.execute('''
                SELECT column_name
                FROM information_schema.columns
                WHERE table_name = %s;''', (table,))

    cnames = pd.DataFrame(cur.fetchall())
    #cnames = [row[0] for row in cur.fetchall()]
    return cnames

HTML(pd.DataFrame(col_names('contratos')).to_html(index=True))

,0
0,id
1,n_anuncio
2,anuncio_preco_base
3,anuncio_proposalDeadline
4,anuncio_cnccs
5,anuncio_contractingProcedureAliasID
6,anuncio_contractingProcedureId
7,anuncio_drPublicationDate
8,anuncio_dreNumber
9,anuncio_dreSeries


    ID's dos diferentes tipos de procedimentos

    1. Ajuste Direto Regime Geral

In [25]:
def ajuste_dir():
    """
    """
    
    cur = conn.cursor()
    
    cur.execute('''
            SELECT "id"
            FROM "contratos"
            WHERE tipo_procedimento = 'Ajuste Direto Regime Geral';''')
    
    return list(cur.fetchall())

    2. Consulta Prévia

In [7]:
def consulta_prev():
    """
    """
    
    cur = conn.cursor()
    
    cur.execute('''
            SELECT "id"
            FROM "contratos"
            WHERE tipo_procedimento = 'Consulta Prévia';''')
    
    return list(cur.fetchall())

    3. Concurso Público

In [26]:
def concurso_pub():
    """
    """
    
    cur = conn.cursor()
    
    cur.execute('''
            SELECT "id"
            FROM "contratos"
            WHERE tipo_procedimento = 'Concurso público';''')
    
    return list(cur.fetchall())

    Restringir por CPV - 72 IT services: consulting, software development, Internet and support

In [19]:
cur = conn.cursor()
cur.execute('''
            SELECT "id"
            FROM "contratos"
            WHERE cpv LIKE '72%';''')

len(cur.fetchall())

187

In [51]:
def cpv_direto(cpv):
    
    """
    Função que retorna os id's de todos os contratos para um determinado CPV para todos os Ajustes Diretos em Regime Geral

    Parâmetros :
        cpv : inserir os dois algarismos iniciais do CPV entre '' ( https://www.publictendering.com/cpv-codes/list-of-the-cpv-codes/ )
    
    Return :
        tuple : id's dos contratos
    """
    
    cur = conn.cursor()
    cur.execute('''
                SELECT "id"
                FROM "contratos"
                WHERE cpv LIKE %s AND tipo_procedimento = 'Ajuste Direto Regime Geral';''', (cpv + '%',))
    
    return cur.fetchall()

ajdir = cpv_direto('72')

In [52]:
def cpv_cpub(cpv):

    """
    Função que retorna os id's de todos os contratos para um determinado CPV para todos os Concursos Públicos

    Parâmetros :
        cpv : inserir os dois algarismos iniciais do CPV entre '' ( https://www.publictendering.com/cpv-codes/list-of-the-cpv-codes/ )
    
    Return :
        tuple : id's dos contratos
    """
    
    cur = conn.cursor()
    cur.execute('''
                SELECT "id"
                FROM "contratos"
                WHERE cpv LIKE %s AND tipo_procedimento = 'Concurso público';''', (cpv + '%',))

    
    return cur.fetchall()

cpub = cpv_cpub('72')

    Número de Contratos


In [53]:
def n_contracts(table):
    '''
    Retorna o número de contratos de uma tabela pertencente à base de dados

    Parâmetros : 
        - table : tabela de interesse

    return : 
        - int : número de contrato
    '''
    
    cur = conn.cursor()
    cur.execute('''
                SELECT COUNT(*) 
                FROM "{}"; '''.format(table))
    ncontract = cur.fetchone()[0]
    return ncontract

n_contracts("contratos")

5537

    Função que retorna contrato para número de id

In [12]:
def contrato(ide):
    '''
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT *
        FROM "contratos"
        WHERE id = %s; ''', (ide,))
    return pd.DataFrame(cur.fetchall())


def contratos(ide):
    '''
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT *
        FROM "contratos"
        WHERE id IN %s; ''', (tuple(ide),))
    return pd.DataFrame(cur.fetchall())

    Preço contratual

In [62]:
def preco_contrato1(ide):
    '''
    Função que retorna preço contratual a partir do id do anúncio para a tabela "contratos"

    Parâmetros :
        - ide : id do anúncio

    Return : 
        - int : preço contratual
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT preco_contratual
        FROM "contratos"
        WHERE id = %s; ''', (ide,))
    return float(cur.fetchone()[0])



# ---------------------------------------------------------------------------------------------------------------------------------------------------------



def preco_contrato2(ide, table = ""):
    '''
    Função que retorna preço contratual a partir do id do anúncio para um dada tabela

    Parâmetros :
        - ide : id do anúncio
        - table : tabela de interesse. Caso input esteja vazio, usar tabela 'contratos'

    Return : 
        - int : preço contratual
    '''

    if table == "": 
        table = "contratos"
        
    cur = conn.cursor()
    cur.execute('''
        SELECT preco_contratual
        FROM "{}"
        WHERE id = %s; '''.format(table), (ide,))
    return float((cur.fetchone())[0])


    
# ---------------------------------------------------------------------------------------------------------------------------------------------------------
    


#def preco_contrato3(id_anuncio):
#    
#    '''
#    Função que retorna preço contratual a partir de uma lista de ids de anúncios
#    '''
#    
#    cur.execute('''
#        SELECT preco_contratual
#        FROM "contratos"
#        WHERE id IN %s; ''', (tuple(id_anuncio),))
#    
#    return np.array(cur.fetchall()[0])

def preco_contrato3(id_anuncio):
    
    '''
    Função que retorna preço contratual a partir de uma lista de ids de anúncios
    '''
    
    cur.execute('''
        SELECT preco_contratual
        FROM "contratos"
        WHERE id IN %s; ''', (tuple(id_anuncio),))

   #preco = list(cur.fetchall())
   #
   #n = len(preco)
   #p = np.zeros(n)

   #for i in range(n):
   #    p[i] = (preco[i][0]).replace(".", "").replace(",",".").replace("€","")

    p = np.array(cur.fetchall(), dtype = float).reshape(len(id_anuncio),)
    return p


    
# ---------------------------------------------------------------------------------------------------------------------------------------------------------
    



def preco_contrato4(ide, table = ""):
    '''
    Função que retorna preço contratual a partir de uma lista de ids de anúncios para uma determinada tabela

    Parâmetros :
        - ide : id do anúncio
        - table : tabela de interesse. Caso input esteja vazio, usar tabela 'contratos'

    Return : 
        - int : preço contratual
    '''

    
    if table == "": 
        table = "contratos"

    
    cur.execute('''
        SELECT preco_contratual
        FROM "{}"
        WHERE id IN %s; '''.format(table), (tuple(ide),))
    
    return np.asarray(cur.fetchall())
    

    
# ---------------------------------------------------------------------------------------------------------------------------------------------------------

In [63]:
preco_contrato3(ids)

array([11900.  , 10409.85, 14530.4 , ..., 12692.  , 24834.5 , 60577.67])

    Preço Base

In [64]:
def preco_base(ide):
    '''
    Função que retorna preço base a partir do id do anúncio para a tabela "contratos"

    Parâmetros :
        - ide : id do anúncio

    Return : 
        - tuplo : preço contratual
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT anuncio_preco_base
        FROM "contratos"
        WHERE id = %s; ''', (ide,))
    
    return cur.fetchone()



def preco_base1(ide):
    '''
    Função que retorna preço base a partir do id do anúncio para a tabela "contratos"

    Parâmetros :
        - ide : id do anúncio

    Return : 
        - int : preço contratual
    '''
        
    cur = conn.cursor()
    cur.execute('''
        SELECT anuncio_preco_base
        FROM "contratos"
        WHERE id = %s; ''', (ide,))


    preco = cur.fetchone()[0]
    p1 = float(preco[:-2].replace(".", "").replace(",","."))
    
    return p1



# ---------------------------------------------------------------------------------------------------------------------------------------------------------



def preco_base2(ide, table = ""):
    '''
    Função que retorna preço base a partir do id do anúncio para um dada tabela

    Parâmetros :
        - ide : id do anúncio
        - table : tabela de interesse. Caso input esteja vazio, usar tabela 'contratos'

    Return : 
        - int : preço contratual
    '''

    if table == "": 
        table = "contratos"


    cur = conn.cursor()
    cur.execute('''
        SELECT anuncio_preco_base
        FROM "{}"
        WHERE id = %s; '''.format(table), (ide,))


    preco = cur.fetchone()[0]

    if preco != 0:
        p1 = float(preco[:-2].replace(".", "").replace(",","."))
    else:
        p1 = 0
    
    return p1
    


# ---------------------------------------------------------------------------------------------------------------------------------------------------------
    


def preco_base3(id_anuncio):
    
    '''
    Função que retorna preço base a partir de uma lista de ids de anúncios
    '''
    
    cur.execute('''
        SELECT anuncio_preco_base
        FROM "contratos"
        WHERE id IN %s; ''', (tuple(id_anuncio),))

    preco = list(cur.fetchall())


    # Como os valores do preco base estão no formato ---.---,--€ é precio converter em -------.-- para comparar posteriormente
    n = len(preco)
    p = np.zeros(n)
    
    for i in range(n):

        if preco[i][0] != 'None':
            p[i] = (preco[i][0]).replace(".", "").replace(",",".").replace("€","")

        else:
            pass
            
    return p

In [65]:
# Conjunto de todos os preços base
a = preco_base3(ids)

#pd.DataFrame(a).describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))
print("Nr de Zero Values :", len(a) - (np.count_nonzero(a)))
print("Nr de Non Zero Values :", (np.count_nonzero(a)))
print("% non zero values : ",round(np.count_nonzero(a)/len(a) * 100,1))

Nr de Zero Values : 2411
Nr de Non Zero Values : 497
% non zero values :  17.1


In [66]:
#nn_index = np.where(a != 0)[0]                 # índices dos ids dos contratos com preço base não nulo
#
#id1 = np.array(ids)                            # Transformação da lista de ids em array. Assim podemos extrair os ids não nulos usando os índices acima
#
#id2 = id1[nn_index]
#id2 = id2.reshape(len(id2),1)
#
#ids_nn = tuple(map(tuple, id2))                # ids não nulos. É preciso converter em tuplo para usar como input na função de preço base
#
#pb = preco_base3(ids_nn)
#pc = preco_contrato3(ids_nn)

In [77]:
df1 = contratos(ids).iloc[70:100,[0,2,18]]
df1=df1.rename(columns = {0:'ID',2:'PrecoBase', 18:'PrecoContratual'})
df1

,ID,PrecoBase,PrecoContratual
70,10322236,None,5088.00
71,10330205,None,8890.00
72,10323553,None,2306.96
73,10323551,None,10000.00
74,10323290,None,1420.32
75,10323268,None,84000.00
76,10323243,None,120000.00
77,10322234,85144.85,69831.80
78,10325296,369261.00,75557.00
79,10323534,46782.75,38160.00


In [68]:
def redflag(pbase, pcontr, tol, ids):
    
    """
    Função que calcula a diferença entre o preço base e preço contratual de um contrato realizado
    Se o preço contratual estiver contido num intervalo em torno do preço base é levantada um flag
    O intervalo é definido pelo parâmetro tolerância e é definido como : [preço_base - preço_base*tolerância, preço_base + preço_base*tolerância]
    Se o preço base não estiver definido, é levantada uma flag também 

    Parâmetros de entrada : 
        pbase : array com os preços base
        pcontr : array com os preços contratuas
        tol : valor da tolerância. Só pode tomar valores entre 0 e 1
        ids : id's dos contratos em questão

    Return : 
        f : tuplo com os id's dos contratos com flag associada
    """

    # Garantir que dimensão dos arrays com os preços é igual
    if len(pbase) != len(pcontr):
        return "Error : dim pbase != dim pcontr"

    # Garantir que tolerância é um numero entre 0 e 1
    if tol < 0 or tol > 1 :
        return "Error : tolerance must belong between 0 and 1"

    # Número de preços base
    n = len(pbase)

    # Array que guarda ocorrência - ou não - de uma flag
    flags = np.zeros(n)

    for i in range(n):

        # Definir limites superior e inferior, respetivamente
        up_lim = pbase[i] * (1 + tol)
        lo_lim = pbase[i] * (1 - tol)

        #if lo_lim <= pcontr[i] <= up_lim :
        #    flags[i] = 1
        #    print('True', pbase[i], pcontr[i])
        #    
        #elif pbase[i] == 0 :
        #    flags[i] = 1    
        #    
        #print(up_lim, lo_lim, pbase[i], pcontr[i])

        if pbase[i] == 0:
            flags[i] = 1

        elif pcontr[i] >= lo_lim :
            print('True')

        if pbase[i] != 0:
            print(lo_lim, up_lim, pcontr[i], pbase[i])

    # Conversão do tuplo de ids num array de uma coluna
    ids = np.array(ids).reshape((n,))

    # Contratos com ocorrência de uma flag
    pos = np.where(flags != 0)

    # Selecionar contratos onde ocorre flag
    f = ids[pos]

    # Conversão do conjunto de contratos em tuplo para poder usar como input nas funções que têm como input id's de contratos
    f = f.reshape((len(f),1))
    f = tuple(map(tuple,f))
    
    return f

pb = preco_base3(ids)
pc = preco_contrato3(ids)
coiso = redflag(pb,pc,0.25, ids)

df = contratos(coiso).head(50).iloc[:,[0,2,18]]
df = df.rename(columns = {0:'ID',2:'PrecoBase', 18:'PrecoContratual'})
df

7205400.0 12009000.0 10409.85 9607200.0
2591133.75 4318556.25 14530.4 3454845.0
180000000.0 300000000.0 2794889.61 240000000.0
5250000.0 8750000.0 58226.0 7000000.0
26202599.25 43670998.75 343898.9 34936799.0
18750000.0 31250000.0 196164.0 25000000.0
53850000.0 89750000.0 93000.0 71800000.0
34275000.0 57125000.0 457000.0 45700000.0
79957545.0 133262575.0 892309.45 106610060.0
15435000.0 25725000.0 178999.0 20580000.0
11250000.0 18750000.0 142641.0 15000000.0
8499675.0 14166125.0 12080.0 11332900.0
14931750.0 24886250.0 12184.0 19909000.0
15794089.5 26323482.5 49993.21 21058786.0
6385863.75 10643106.25 69831.8 8514485.0
27694575.0 46157625.0 75557.0 36926100.0
3508706.25 5847843.75 38160.0 4678275.0
7725000.0 12875000.0 49618.0 10300000.0
14931750.0 24886250.0 67623.41 19909000.0
11550000.0 19250000.0 20194.25 15400000.0
10732500.0 17887500.0 556.0 14310000.0
8231293.5 13718822.5 12917.28 10975058.0
2591133.75 4318556.25 2837.81 3454845.0
2141280.0 3568800.0 25301.28 2855040.0
7205400.0

,ID,PrecoBase,PrecoContratual
0,10322372,None,11900.00
1,10323701,None,8932.00
2,10323697,None,12000.00
3,10323706,None,13860.00
4,10323705,None,10000.00
5,10323359,None,801.00
6,10323281,None,15000.00
7,10322358,None,8000.00
8,10323698,None,9920.00
9,10325356,None,1780.00


<br>

<br>

<br>

<br>

<br>

In [83]:
cur = conn.cursor()
cur.execute('''
        SELECT tipo_procedimento, COUNT (tipo_procedimento)
        FROM "contratos"
        GROUP BY "tipo_procedimento"
        ORDER BY COUNT(tipo_procedimento) DESC;;''')

df = (pd.DataFrame(cur.fetchall()))
HTML(df.to_html(index=False))

0,1
Ajuste Direto Regime Geral,773
Consulta Prévia,253
Ao abrigo de acordo-quadro (art.º 259.º),234
Concurso público,203
Ao abrigo de acordo-quadro (art.º 258.º),36
Consulta Prévia Simplificada,6
Concurso limitado por prévia qualificação,4
Procedimento de negociação,2


In [92]:
cur = conn.cursor()

cur.execute('''
        SELECT "contractTypes", COUNT ("contractTypes")
        FROM "contratos"
        GROUP BY "contractTypes"
        ORDER BY  COUNT("contractTypes") DESC;;''')

df = (pd.DataFrame(cur.fetchall()))
HTML(df.to_html(index=False))

0,1
Aquisição de bens móveis,836
Aquisição de serviços,535
Empreitadas de obras públicas,118
Locação de bens móveis,8
Aquisição de bens móveis<br/>Aquisição de serviços,6
Aquisição de serviços<br/>Aquisição de bens móveis,3
Concessão de obras públicas,2
Concessão de serviços públicos,1
Aquisição de serviços<br/>Empreitadas de obras públicas<br/>Aquisição de bens móveis,1
Aquisição de serviços<br/>Empreitadas de obras públicas,1


In [7]:
cur.execute('''
    TABLE "contratos";
''')

df = (pd.DataFrame(cur.fetchmany(1)))
HTML(df.to_html(index=False))

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
10323526,9385/2023,"1.893.000,00 €",25 dias.,False,6549928,13118913,06-06-2023,109,2,False,False,https://dre.pt/application/file/214106482,Anúncio de procedimento,Concurso público,Requalificação e Ampliação das Piscinas Municipais de Fânzeres,02-11-2023,16-10-2023,1863547.21,366,Município de Gondomar (506848957)(https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=173),"Artigo 19.º, alínea b) do Código dos Contratos Públicos","Ivo Ribeiro, Unipessoal, Lda. (Electroinstal) (507655745)(https://www.base.gov.pt/Base4/pt/detalhe/?type=entidades&id=32793)","Arpecdouro S.A (508263425)|||Edilages, S.A. (508559871)|||Décio Soares , Unipessoal, Lda. (509274374)",https://www.base.gov.pt/Base4/pt/detalhe/?type=anuncios&id=348901,https://community.vortal.biz/PRODPublic/Tendering/OpportunityDetail/Index?noticeUID=PT1.NTC.2420642,https://www.base.gov.pt/Base4/pt/resultados/?type=doc_documentos&id=2071195&ext=.pdf,false,45453000-7,False,None,None,False,False,None,False,None,False,Empreitadas de obras públicas,Obras de revisão e recuperação,Principal,1863547.21 €,Requalificação e Ampliação das Piscinas Municipais de Fânzeres,Não aplicável,None,366 dias,"Portugal, Porto, Gondomar",Não aplicável.,Não aplicável.,None,False,False,None,None,False,,True,None,"Código dos Contratos Públicos (DL111-B/2017) e Lei n.º 30/2021, de 21.05",None,None,1699032847
